# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

#### 1.) What test is appropriate for this problem? Does CLT apply?

For CLT to apply,the  samples have to be independent and  the product of sample size and probability of occurrence has to be greater than or equal to 5

In [28]:
p1 = sum(data.call[data.race=='w'])/sum(data.race=='w')

n1=sum(data.race=='w')






n2 = sum(data.race=='b')
p2 = sum(data.call[data.race=='b'])/sum(data.race=='b')

popu_call_diff = p1-p2

#combined proportion value, cp
cp = (n1*p1 + n2*p2) / (n1+n2)
std_dev = np.sqrt(cp*(1-cp)/n1 + cp*(1-cp)/n2)

print(n1*p1, n1*(1-p1), n2*p2 , n2*(1-p2))



235.0 2200.0 157.0 2278.0


It can be seen that CLT applies as the above stated conditions for CLT are satisfied

#### 2) What are the null and alternate hypotheses?

Null hypothesis is that  there is no racial discrimination in call back and that difference in call back rates is purely co-incidental.
Alternate hypothesis is that there is significant impact of race on the callback rates.

In [11]:
w = data[data.race=='w']
b = data[data.race=='b']
white_call = data[data.race=='w'].call
black_call = data[data.race=='b'].call

#### 3.) Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.


In [12]:
# Bootstrapping


def prop_call_diff(data1, data2):
    p1 = sum(data1['call'=='1'])/len(data1)
    p2 = sum(data2['call'=='1'])/len(data2)
    return abs (p1-p2)



def bs_samples_diff(data1, data2, size):
    len1 = len(data1)
    len2 = len(data2)
    perm_reps = np.empty(size)   
   
    
    for i in range(size):
        common = np.concatenate((data1,data2))
        common_perm = np.random.permutation(common)
        perm_rep_b = common_perm[: len1]
        perm_rep_w = common_perm[len1:]
        perm_reps[i] = prop_call_diff(perm_rep_b,perm_rep_w) 
       
    return perm_reps


sample_call_diff = bs_samples_diff(black_call,white_call,10000)


# pvalue

p = np.sum(sample_call_diff > popu_call_diff)/len(sample_call_diff)
print('On doing bootstrapping, number of times the randomly selected prop differences is greater than our samples prop diff is: ', p)


On doing bootstrapping, number of times the randomly selected prop differences is greater than our samples prop diff is:  0.0


The p-value for the Null hypothesis is 0.0, which implies that the NUll hypothesis tha there is no racial discrimination in call back and that difference in call back rates is purely co-incidental has FAILED.
Alternate hypothesis  that there is significant impact of race on the callback rates seems to be the case.

In [34]:
#Frequentist approach
# Margin of Error, Confidence Interval  and p value 

MoE = 1.96 * std_dev
print('Margin of Error=', MoE)

# t and p values
t,p = stats.ttest_ind(white_call, black_call, equal_var = False)
print('t-stat=',t, 'p-value', p)

# confidence interval

print('Confidence Interval', popu_call_diff+MoE,popu_call_diff-MoE)




Margin of Error= 0.015281912310894095
t-stat= 4.114705290861751 p-value 3.942941513645935e-05
Confidence Interval 0.04731476652033968 0.01675094189855149


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

####  4). Write a story describing the statistical significance in the context or the original problem.

Racial discrimination has existed in various forms throughout much of recorded history.
Here the investigation of the distribution of the  data set shows that the racial component is prevailing when it comes to callback for job.

The p-value found is well below the threshold of 0.05 (means it has a statistical significance), so the null hypothesis can be rejected. And as per data wecan say that race was considered for a job callback.This shows that the race of a candidate does have a statistically significant effect on his/her chance of getting a callback.

####  5). Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, this analysis does not imply that race/ name is the most important factor in callback success.
To deduce that inference, it would be necessary to do a similar comparison with  other influecning factors like gender, social background, educational qualification, previous experience, etc